# This notebook calculates a signal to noise ratio

It can calculate a S/N based on a star and a telescope property.

This is a script to calculate the sensitivity of a telescope to detect microlensing exoplanet's signal


In [5]:
import numpy as np
from astropy import constants as const
from astropy import units as u
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show

# Function for plotting

In [6]:

from bokeh.models import Whisker, ColumnDataSource, Span, BoxAnnotation
from bokeh.plotting import figure, show


def plotter(x_axis, y_axis, y_error, p, legend_label='', x_label_name='Days', y_label_name='Magnification', color='purple',
            plot_errorbar=False, t0_error_plot=False, t0=None, t0_error=None, type_plot='circle',
            plot_baseline=False, legend_location="bottom_center"):
    """
    Produce plot for the event
    :return: None
    """

    p.xaxis.axis_label = x_label_name
    p.yaxis.axis_label = y_label_name

    if type_plot == 'line':
        p.line(x_axis, y_axis, line_width=2, line_alpha=1.0, legend_label=legend_label, color=color)
    else:
        p.circle(x_axis, y_axis, fill_alpha=0.7, size=5, legend_label=legend_label, color=color)

    if plot_errorbar:
        upper = [x + e for x, e in zip(y_axis, y_error)]
        lower = [x - e for x, e in zip(y_axis, y_error)]
        source = ColumnDataSource(data=dict(groups=x_axis, counts=y_axis, upper=upper, lower=lower))
        whisker_errorbar = Whisker(source=source, base="groups", upper="upper", lower="lower",
                                   line_width=1.0, line_color=color) #level="overlay",
        whisker_errorbar.upper_head.line_color = color
        whisker_errorbar.lower_head.line_color = color
        p.add_layout(whisker_errorbar)
        #p.legend.glyph_height =  20
        #p.legend.glyph_width =  # some int


    if t0_error_plot:
        t0_location = Span(location=t0,
                           dimension='height', line_color='red',
                           line_dash='dashed', line_width=1)
        p.add_layout(t0_location)

        box = BoxAnnotation(left=(t0 - t0_error), right=(t0 + t0_error),
                            line_width=1, line_color='black', line_dash='dashed',
                            fill_alpha=0.2, fill_color='orange')

        p.add_layout(box)

    if plot_baseline:
        horizontal_line = Span(location=0, dimension='width', line_color='grey', line_width=2, line_alpha=0.8)
        p.add_layout(horizontal_line)

    p.legend.background_fill_alpha = 0.0
    p.legend.location = legend_location
    p.legend.label_text_font_size = '8pt'

    return p

# Main functions for sensitivy/signal to noise ratio

In [7]:
def print_cyan(to_be_printed):
    print("\033[96m {}\033[00m".format(to_be_printed))


def planck_einstein_relation(wavelength):
    """
    This function calculates the energy of a photon in a specific wavelength
    :param wavelength in meter
    :return: energy_photon in joule
    """
    planck_constant = const.h
    speed_of_light = const.c
    energy_photon = (planck_constant * speed_of_light) / wavelength
    return energy_photon


def photons_emitted_by_a_star_per_second(luminosity, energy_photon):
    """
    This function calculates how many photons are emitted from a star according to its luminosity
    :param luminosity:
    :param energy_photon:
    :return: number_photons
    """
    number_photons = luminosity / energy_photon
    return number_photons * photons


def photons_arriving_to_a_certain_distance(number_photons, distance):
    """
    how many photons will arrive to this region far from the star
    :param number_photons:
    :param distance:
    :return:
    """
    photons_arriving = number_photons / (4 * np.pi * distance ** 2)
    return photons_arriving


def photons_after_telescope_aperture(number_photons_per_area, aperture, type='round'):
    """
    :param number_photons_per_area:
    :param aperture:
    :param type:
    :return:
    """
    if type == 'round':
        number_photons = number_photons_per_area * (4 * np.pi * (aperture/2) ** 2)
    else:
        number_photons = number_photons_per_area * (aperture ** 2)
    return number_photons


def photoelectrons_no_amplification(photons_getting_in, quantum_efficiency, etenue):
    """
    This function consider the quantum efficiency and the etenue to see how many photoelectrons are generated when
    a specific number os photons come in the telescope.
    :param photons_getting_in:
    :param quantum_efficiency:
    :param etenue:
    :return: number_photoelectrons
    """
    number_photoelectrons = (photons_getting_in/photons) * quantum_efficiency * etenue
    return number_photoelectrons * photoelectrons


def photoelectrons_with_amplification(number_photoelectrons, amplification):
    """
    Gravitational microlensing works like a lens. The lens star curves the space-time, bending the light from the
    back source star. Therefore, more photons come in, then, more photoelectrons are generated.
    :param number_photoelectrons:
    :param amplification:
    :return: more_photoelecrons
    """
    more_photoelecrons = number_photoelectrons * amplification
    return more_photoelecrons


def signal_to_noise_ratio(photoelectrons_per_second_signal, dark_current_noise, read_out_noise, diffuse_background):
    """
    This function calculated the signal to noise, using as input the count of photoelectrons per second for all of these
    signal, dark_current_noise, read_out_noise, and diffuse_background.
    :param photoelectrons_per_second_signal:
    :param dark_current_noise:
    :param read_out_noise:
    :param diffuse_background:
    :return: signal to noise ratio
    """
    # This is the formula found in the paper. but it is weird
    # snr = photoelectrons_per_second_signal / np.sqrt(photoelectrons_per_second_signal + dark_current_noise +
    #                                                  read_out_noise + diffuse_background)
    # so I wll try with the square
    snr = photoelectrons_per_second_signal / np.sqrt(photoelectrons_per_second_signal**2 + dark_current_noise**2 +
                                                     read_out_noise**2 + diffuse_background**2)

    return snr


def relative_signal_to_noise_ratio_calculator(snr_base, snr_detection):
    """
    this function calculates the relative signal to noise ratio
    :param snr_base:
    :param snr_detection:
    :return:
    """
    relative_snr = (snr_detection - snr_base)/snr_base
    return relative_snr


def command_to_return_relative_signal_to_noise_ratio(wavelength_star_peak, luminosity_of_the_star,
                                                     distance_from_observer, telescope_diameter,
                                                     quantum_efficiency_value, etenue_value, dark_current,
                                                     read_out, diffuse_background,
                                                     magnification):
    """
    This function commands the other functions to give us the relative signal to noise ratio
    :param wavelength_star_peak:
    :param luminosity_of_the_star:
    :param distance_from_observer:
    :param telescope_diameter:
    :param quantum_efficiency_value:
    :param etenue_value:
    :param dark_current:
    :param read_out:
    :param diffuse_background:
    :param magnification:
    :return: relative signal to noise ratio
    """

    # ==================================================================================================================
    #                                            RUNNING MAIN CODE
    # ==================================================================================================================
    energy_of_a_photon = planck_einstein_relation(wavelength_star_peak)
    number_emitted_photons_per_second = photons_emitted_by_a_star_per_second(luminosity_of_the_star, energy_of_a_photon)

    photons_from_a_certain_distance = photons_arriving_to_a_certain_distance(number_emitted_photons_per_second,
                                                                             distance_from_observer)

    number_photons_getting_in = photons_after_telescope_aperture(photons_from_a_certain_distance, telescope_diameter)

    generated_photoelectrons = photoelectrons_no_amplification(number_photons_getting_in,
                                                               quantum_efficiency_value, etenue_value)

    photoelectrons_when_lensed = photoelectrons_with_amplification(generated_photoelectrons, magnification)

    signal_to_noise_ratio_no_microlensing = signal_to_noise_ratio(generated_photoelectrons,
                                                                  dark_current, read_out, diffuse_background)

    signal_to_noise_ratio_microlensing = signal_to_noise_ratio(photoelectrons_when_lensed,
                                                               dark_current, read_out, diffuse_background)

    relative_signal_to_noise_ratio = relative_signal_to_noise_ratio_calculator(signal_to_noise_ratio_no_microlensing,
                                                                    signal_to_noise_ratio_microlensing)
    # ==================================================================================================================
    #                          PRINTING INTERMEDIARY STEPS JUST TO FOLLOW WHAT IS GOING ON
    # ==================================================================================================================
    print(f"\nNumber of emitted photons by a star of \nluminosity {luminosity_of_the_star} and \nwavelength peak "
          f"{wavelength_star_peak} \nper second:")
    print_cyan(number_emitted_photons_per_second.to(photons / u.s))
    print(f"\nNumber of photons arriving to an observer {distance_from_observer} away per area and second: ")
    print_cyan(photons_from_a_certain_distance.to(photons / (u.cm ** 2 * u.s)))

    print(f"\nNumber of photons getting in the telescope of aperture {telescope_diameter} in diameter: ")
    print_cyan(number_photons_getting_in.to(photons / u.s))

    print(f"\nNumber of photoelectrons generated when the quantum efficiency is {quantum_efficiency_value}"
          f"and the etenue is {etenue_value}: ")
    print_cyan(generated_photoelectrons.to(photoelectrons / u.s))

    print(f"\nNumber of photoelectrons generated when there is an amplification of {magnification}")
    print_cyan(photoelectrons_when_lensed.to(photoelectrons / u.s))

    print(
        f"\nAssuming dar current of {dark_current}, read noise of {read_out} and background count of {diffuse_background}")
    # print(f"Signal to Noise ratio without microlensing")
    # print_cyan((signal_to_noise_ratio_no_microlensing.to((photoelectrons / u.s) ** (1 / 2))).value)
    # print(f"Signal to Noise ratio with microlensing of {magnification} magnification")
    # print_cyan((signal_to_noise_ratio_microlensing.to((photoelectrons / u.s) ** (1 / 2))).value)
    print(f"Signal to Noise ratio without microlensing")
    print_cyan(signal_to_noise_ratio_no_microlensing.value)
    print(f"Signal to Noise ratio with microlensing of {magnification} magnification")
    print_cyan(signal_to_noise_ratio_microlensing.value)
    print("Relative signal to noise (snr_microlensing - snr_base)/snr_base")
    print_cyan(relative_signal_to_noise_ratio)
    return signal_to_noise_ratio_no_microlensing, signal_to_noise_ratio_microlensing, relative_signal_to_noise_ratio


def plot_relative_signal_to_noise_ratio_in_function_of(in_function_of, relative_snr, string_in_function_of,
                                                       fixed_param, color, p1):

    ## PLotting model
    p1 = plotter(in_function_of, relative_snr, [], p1, legend_label=f'{fixed_param}',
                 x_label_name='Diameter aperture', y_label_name='Signal to Noise Ratio',
                 color=color, plot_errorbar=False, t0_error_plot=False, t0=None, t0_error=None, type_plot='line',
                 legend_location="top_left")
    return p1




# Setting the assumptions

In [8]:
# ==================================================================================================================
#                              SETTING SOME PHOTONS AND PHOTOELECTRONS AS UNIT
# ==================================================================================================================
# "photons" unit definition
photons = u.def_unit('photons')
# "photoelectrons" unit definition
photoelectrons = u.def_unit('photoelectrons')


# ==================================================================================================================
#                                            SET YOUR PARAMETERS HERE
# ==================================================================================================================
# Star properties
# TODO: change to K' band , 
# (center: 2120nm), (cut on: 1950nm) (cut off: 2290nm)
wavelength_star_peak = np.float(4e-7) * u.meter # for red stars, sun in green: np.float(550e-9) * u.meter
# Red Giant Star
luminosity_of_the_star = 1000 *np.float(3.828e26) * u.watt  # The sun: 3.828e26 W
                                                            # Red-giant-branch stars have luminosities up to nearly three
                                                            # thousand times that of the Sun. I assumed 1000*L_sun

# Telescope properties
distance_from_observer = np.float(10e0) * u.lightyear
telescope_diameter = np.float(15e0) * u.cm
quantum_efficiency_value = 0.95
etenue_value = 0.95
dark_current = 0.001 * photoelectrons/u.second  # (e-/s)
read_out = 3.0 * photoelectrons/u.second  # This do not seem right.
                                        # Everything for read out seems not to be /time (divided)
diffuse_background = 1.0 * photoelectrons/u.second

# Gravitational Lenses parameters:
magnification = 1.34

signal_to_noise_ratio_no_microlensing, signal_to_noise_ratio_microlensing, relative_signal_to_noise_ratio = \
    command_to_return_relative_signal_to_noise_ratio(
                                                     wavelength_star_peak, luminosity_of_the_star,
                                                     distance_from_observer, telescope_diameter,
                                                     quantum_efficiency_value, etenue_value, dark_current,
                                                     read_out, diffuse_background,
                                                     magnification)

# # Code varying the telescope diameter and the distance from the observer
# telescope_diameter = np.arange(8e0, 30e0) * u.cm

# distance_from_observer_s = [np.float(1.5e3) * u.parsec, np.float(3e3) * u.parsec,  np.float(4e3) * u.parsec,
#                             np.float(5e3) * u.parsec, np.float(6e3) * u.parsec, np.float(7e3) * u.parsec]
# colors = ['black', 'maroon', 'blueviolet', 'skyblue', 'green', 'orange']

# output_file(f'relative_signal_to_noise_ratio_in_function_of_telescope_diameter.html')

# p1 = figure(title=f"Relative signal to noise ratio in function of telescope diameter",
#             plot_width=900, plot_height=500)

# for distance_from_observer, color in zip(distance_from_observer_s, colors):
#     signal_to_noise_ratio_no_microlensing, signal_to_noise_ratio_microlensing, relative_signal_to_noise_ratio = command_to_return_relative_signal_to_noise_ratio(wavelength_star_peak, luminosity_of_the_star,
#                                                       distance_from_observer, telescope_diameter,
#                                                       quantum_efficiency_value, etenue_value, dark_current,
#                                                       read_out, diffuse_background,
#                                                       magnification)

#     p1 = plot_relative_signal_to_noise_ratio_in_function_of(telescope_diameter.value, signal_to_noise_ratio_no_microlensing,
#                                                         'Telescope Diameter', f'Observer {distance_from_observer} away',
#                                                         color, p1)
output_notebook()
show(p1)


Number of emitted photons by a star of 
luminosity 8.000000000000001e+24 W and 
wavelength peak 4e-07 m 
per second:
 1.610917301613667e+43 photons / s

Number of photons arriving to an observer 10.0 lyr away per area and second: 
 14322.342065166995 photons / (cm2 s)

Number of photons getting in the telescope of aperture 15.0 cm in diameter: 
 10123867.038178958 photons / s

Number of photoelectrons generated when the quantum efficiency is 0.95and the etenue is 0.95: 
 9136790.001956508 photoelectrons / s

Number of photoelectrons generated when there is an amplification of 1.34
 12243298.602621723 photoelectrons / s

Assuming dar current of 0.001 photoelectrons / s, read noise of 3.0 photoelectrons / s and background count of 1.0 photoelectrons / s
Signal to Noise ratio without microlensing
 0.9999999999999402
Signal to Noise ratio with microlensing of 1.34 magnification
 0.9999999999999667
Relative signal to noise (snr_microlensing - snr_base)/snr_base
 2.653433028854283e-14


# TODO: K' passband

I found that red-giant-branch stars have luminosities up to nearly three thousand times that of the Sun. I assumed 1000*L_sun. 

Should I be looking to the black-body radiation curve to integrate something? Instead of peaking a luminosity X ? 



# TODO:  S/N to error bar
How do I get the value for S/N found here to use as my error in the simulated light curve?